In [8]:
from keras.applications import MobileNet

In [9]:
img_rows, img_cols = 224, 224

mobilenet_model = MobileNet(weights = 'imagenet',
                            include_top = False,
                            input_shape = (img_rows, img_cols, 3)
                           
                           )

for layers in mobilenet_model.layers:
    layers.trainable = False
    
for i, layer in enumerate(mobilenet_model.layers):
    print(str(i) + ' ' + layer.__class__.__name__, layer.trainable)

0 InputLayer False
1 ZeroPadding2D False
2 Conv2D False
3 BatchNormalization False
4 ReLU False
5 DepthwiseConv2D False
6 BatchNormalization False
7 ReLU False
8 Conv2D False
9 BatchNormalization False
10 ReLU False
11 ZeroPadding2D False
12 DepthwiseConv2D False
13 BatchNormalization False
14 ReLU False
15 Conv2D False
16 BatchNormalization False
17 ReLU False
18 DepthwiseConv2D False
19 BatchNormalization False
20 ReLU False
21 Conv2D False
22 BatchNormalization False
23 ReLU False
24 ZeroPadding2D False
25 DepthwiseConv2D False
26 BatchNormalization False
27 ReLU False
28 Conv2D False
29 BatchNormalization False
30 ReLU False
31 DepthwiseConv2D False
32 BatchNormalization False
33 ReLU False
34 Conv2D False
35 BatchNormalization False
36 ReLU False
37 ZeroPadding2D False
38 DepthwiseConv2D False
39 BatchNormalization False
40 ReLU False
41 Conv2D False
42 BatchNormalization False
43 ReLU False
44 DepthwiseConv2D False
45 BatchNormalization False
46 ReLU False
47 Conv2D False
48 Batc

In [10]:
def addTopModel(bottom_model, num_classes):
    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(1024, activation = 'relu')(top_model)
    top_model = Dense(num_classes, activation = 'softmax')(top_model)
    return top_model

In [11]:
import tensorflow as tf
from keras import Sequential, Model
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, Activation, Flatten, Conv2D, MaxPooling2D, ZeroPadding2D, BatchNormalization


In [12]:
num_classes = 10

FC_Head = addTopModel(mobilenet_model, num_classes)

model = Model(inputs = mobilenet_model.input, output = FC_Head)

print(model.summary())

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32)      128 

C:\Users\ASK Computers\Anaconda3\envs\py3-TF2.0\lib\site-packages\ipykernel_launcher.py:5: UserWarning: Update your `Model` call to the Keras 2 API: `Model(inputs=Tensor("in..., outputs=Tensor("de...)`
  """


In [13]:
from keras.preprocessing.image import ImageDataGenerator

In [15]:
train_data_dir = r'C:\Users\ASK Computers\Desktop\CV Projects\DeepLearningCV\monkey_breed\train'
validation_data_dir = r'C:\Users\ASK Computers\Desktop\CV Projects\DeepLearningCV\monkey_breed\validation'

train_datagen = ImageDataGenerator(rescale = 1/ 255.,
                                   rotation_range = 45,
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   horizontal_flip = True,
                                   fill_mode = 'nearest'
                                  )

val_datagen = ImageDataGenerator(rescale = 1/ 255.)

batch_size = 32

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'categorical'
)

val_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (img_rows, img_cols),
    batch_size = batch_size,
    class_mode = 'categorical'

)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [16]:
from keras.callbacks import EarlyStopping, ModelCheckpoint


In [18]:
checkpoint = ModelCheckpoint(
    'Checkpoints/Transfer_Learning_Monkey_Breed_Classifier.h5',
    monitor = 'val_loss',
    mode = 'min',
    save_best_only = True,
    verbose = 1
)

earlystop = EarlyStopping(
    min_delta = 0,
    patience = 3,
    monitor = 'val_loss',
    mode = 'min',
    verbose = 1,
    restore_best_weights = True
)

callbacks = [checkpoint, earlystop]

In [20]:
model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

In [22]:
max_epochs = 10
batch_size = 32

history = model.fit(train_generator, validation_data = val_generator, epochs = max_epochs, callbacks = callbacks)


Epoch 1/10
35/35 [==============================] - 231s 7s/step - loss: 1.3805 - accuracy: 0.5619 - val_loss: 0.3940 - val_accuracy: 0.8529

Epoch 00001: val_loss improved from inf to 0.39398, saving model to Checkpoints/Transfer_Learning_Monkey_Breed_Classifier.h5
Epoch 2/10
35/35 [==============================] - 216s 6s/step - loss: 0.4473 - accuracy: 0.8406 - val_loss: 0.5422 - val_accuracy: 0.9007

Epoch 00002: val_loss did not improve from 0.39398
Epoch 3/10
35/35 [==============================] - 206s 6s/step - loss: 0.4083 - accuracy: 0.8707 - val_loss: 0.1476 - val_accuracy: 0.9375

Epoch 00003: val_loss improved from 0.39398 to 0.14756, saving model to Checkpoints/Transfer_Learning_Monkey_Breed_Classifier.h5
Epoch 4/10
35/35 [==============================] - 206s 6s/step - loss: 0.2876 - accuracy: 0.9053 - val_loss: 1.5261 - val_accuracy: 0.8419

Epoch 00004: val_loss did not improve from 0.14756
Epoch 5/10
35/35 [==============================] - 206s 6s/step - loss: 0.4

In [23]:
from keras.models import load_model

classifier = load_model('Checkpoints/Transfer_Learning_Monkey_Breed_Classifier.h5')

In [24]:
import os
import numpy as np
import cv2
from os.path import isfile, join
from os import listdir

In [25]:
monkey_breeds_dict = {"[0]": "mantled_howler ", 
                      "[1]": "patas_monkey",
                      "[2]": "bald_uakari",
                      "[3]": "japanese_macaque",
                      "[4]": "pygmy_marmoset ",
                      "[5]": "white_headed_capuchin",
                      "[6]": "silvery_marmoset",
                      "[7]": "common_squirrel_monkey",
                      "[8]": "black_headed_night_monkey",
                      "[9]": "nilgiri_langur"}

monkey_breeds_dict_n = {"n0": "mantled_howler ", 
                      "n1": "patas_monkey",
                      "n2": "bald_uakari",
                      "n3": "japanese_macaque",
                      "n4": "pygmy_marmoset ",
                      "n5": "white_headed_capuchin",
                      "n6": "silvery_marmoset",
                      "n7": "common_squirrel_monkey",
                      "n8": "black_headed_night_monkey",
                      "n9": "nilgiri_langur"}


In [43]:
def draw_test(name, pred, im):
    monkey = monkey_breeds_dict[str(pred)]
    BLACK = [0, 0, 0]
    expanded_image = cv2.copyMakeBorder(im, 80, 0, 0, 100, cv2.BORDER_CONSTANT, value = BLACK)
    cv2.putText(expanded_image, monkey, (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
    cv2.imshow(name, expanded_image)
    
def getRandomImage(path):
    folders = list(filter(lambda x: os.path.isdir(os.path.join(path, x)), os.listdir(path)))
    random_directory = np.random.randint(0, len(folders))
    path_class = folders[random_directory]
    print('Class -' + monkey_breeds_dict_n[str(path_class)])
    file_path = path + path_class
    file_names = [f for f in listdir(file_path) if isfile(join(file_path, f))]
    random_file_index = np.random.randint(0, len(file_names))
    image_name = file_names[random_file_index]
    return cv2.imread(file_path + '\\' + image_name)

for i in range(0, 10):
    input_im = getRandomImage(r'C:\Users\ASK Computers\Desktop\CV Projects\DeepLearningCV\monkey_breed\validation\\')
    input_original = input_im.copy()
    input_original = cv2.resize(input_original, None, fx = 0.5, fy = 0.5, interpolation = cv2.INTER_LINEAR)
    
    input_im = cv2.resize(input_im, (224, 224), interpolation = cv2.INTER_LINEAR)
    input_im = input_im / 255.
    input_im = np.array(input_im).reshape(1, 224, 224, 3)
    
    res = np.argmax(classifier.predict(input_im, 1, verbose = 0), axis = 1)
    
    draw_test('Prediction', res, input_original)
    cv2.waitKey(0)
    
cv2.destroyAllWindows()

Class -pygmy_marmoset 
Class -common_squirrel_monkey
Class -mantled_howler 
Class -bald_uakari
Class -japanese_macaque
Class -patas_monkey
Class -mantled_howler 
Class -patas_monkey
Class -japanese_macaque
Class -black_headed_night_monkey
